In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq

# part i q1

In [2]:
ois = pd.read_excel("IR Data.xlsx", sheet_name = 'OIS', usecols = ["Tenor", "Product", "Rate"])
ois

,Tenor,Product,Rate
0,6m,OIS,0.00250
1,1y,OIS,0.00300
2,2y,OIS,0.00325
3,3y,OIS,0.00335
4,4y,OIS,0.00350
5,5y,OIS,0.00360
6,7y,OIS,0.00400
7,10y,OIS,0.00450
8,15y,OIS,0.00500
9,20y,OIS,0.00525


In [3]:
#tenors = ['6m', '1y', '2y', '3y', '4y', '5y', '7y', '10y', '15y', '20y', '30y']
tenors = ois["Tenor"]

# Define a function to convert tenor strings to days
def tenor_to_days(tenor):
    if 'm' in tenor:
        return int(tenor.replace('m', '')) * 30
    elif 'y' in tenor:
        return int(tenor.replace('y', '')) * 360
    else:
        return None

def tenor_to_years(tenor):
    if 'm' in tenor:
        return int(tenor.replace('m', '')) / 12
    elif 'y' in tenor:
        return int(tenor.replace('y', ''))
    else:
        return None

# Convert each tenor to days
ois["days"] = [tenor_to_days(tenor) for tenor in tenors]
ois["years"] = [tenor_to_years(tenor) for tenor in tenors]
ois

,Tenor,Product,Rate,days,years
0,6m,OIS,0.00250,180,0.5
1,1y,OIS,0.00300,360,1.0
2,2y,OIS,0.00325,720,2.0
3,3y,OIS,0.00335,1080,3.0
4,4y,OIS,0.00350,1440,4.0
5,5y,OIS,0.00360,1800,5.0
6,7y,OIS,0.00400,2520,7.0
7,10y,OIS,0.00450,3600,10.0
8,15y,OIS,0.00500,5400,15.0
9,20y,OIS,0.00525,7200,20.0


In [4]:
ois1 = ois.copy()
ois1[['sum_df','df']] = np.nan
ois1

,Tenor,Product,Rate,days,years,sum_df,df
0,6m,OIS,0.00250,180,0.5,NaN,NaN
1,1y,OIS,0.00300,360,1.0,NaN,NaN
2,2y,OIS,0.00325,720,2.0,NaN,NaN
3,3y,OIS,0.00335,1080,3.0,NaN,NaN
4,4y,OIS,0.00350,1440,4.0,NaN,NaN
5,5y,OIS,0.00360,1800,5.0,NaN,NaN
6,7y,OIS,0.00400,2520,7.0,NaN,NaN
7,10y,OIS,0.00450,3600,10.0,NaN,NaN
8,15y,OIS,0.00500,5400,15.0,NaN,NaN
9,20y,OIS,0.00525,7200,20.0,NaN,NaN


In [5]:
def equation1(df, rate):
    return rate*df - (1 - df)
    
guess = 0.99

ois1.at[1, 'df'] = fsolve(equation1, guess, args=(ois1.at[1, 'Rate']))[0]
ois1.at[1, 'sum_df'] = ois1.at[1, 'df']
ois1

,Tenor,Product,Rate,days,years,sum_df,df
0,6m,OIS,0.00250,180,0.5,NaN,NaN
1,1y,OIS,0.00300,360,1.0,0.997009,0.997009
2,2y,OIS,0.00325,720,2.0,NaN,NaN
3,3y,OIS,0.00335,1080,3.0,NaN,NaN
4,4y,OIS,0.00350,1440,4.0,NaN,NaN
5,5y,OIS,0.00360,1800,5.0,NaN,NaN
6,7y,OIS,0.00400,2520,7.0,NaN,NaN
7,10y,OIS,0.00450,3600,10.0,NaN,NaN
8,15y,OIS,0.00500,5400,15.0,NaN,NaN
9,20y,OIS,0.00525,7200,20.0,NaN,NaN


In [6]:
def equation2(df, rate, ois_df):
    return rate*(df + ois_df.sum()) - (1 - df)
    
for i in range(2, 6):
    ois1.at[i, 'df'] = fsolve(equation2, guess, args=(ois1.at[i, 'Rate'], ois1["df"]))[0]
    ois1.at[i, 'sum_df'] = ois1.at[i, 'df'] + ois1.at[i-1, 'sum_df']
ois1

,Tenor,Product,Rate,days,years,sum_df,df
0,6m,OIS,0.00250,180,0.5,NaN,NaN
1,1y,OIS,0.00300,360,1.0,0.997009,0.997009
2,2y,OIS,0.00325,720,2.0,1.990540,0.993531
3,3y,OIS,0.00335,1080,3.0,2.980555,0.990015
4,4y,OIS,0.00350,1440,4.0,3.966672,0.986117
5,5y,OIS,0.00360,1800,5.0,4.948856,0.982184
6,7y,OIS,0.00400,2520,7.0,NaN,NaN
7,10y,OIS,0.00450,3600,10.0,NaN,NaN
8,15y,OIS,0.00500,5400,15.0,NaN,NaN
9,20y,OIS,0.00525,7200,20.0,NaN,NaN


In [7]:
def equation3(df, rate, i, ois1):
    sum1 = ois1.at[i-1,'sum_df']
    sum2 = (ois1.at[i, 'years']-ois1.at[i-1, 'years']+1)*(df + ois1.at[i-1,'df'])/2 - ois1.at[i-1, 'df']
    return rate*(sum1 + sum2) - (1 - df)

for i in range(6, len(ois1)):
    
    ois1.at[i, 'df'] = fsolve(equation3, guess, args=(ois1.at[i, 'Rate'], i, ois1))[0]

    sum2 = (ois1.at[i, 'years']-ois1.at[i-1, 'years']+1)*(ois1.at[i,'df'] + ois1.at[i-1,'df'])/2 - ois1.at[i-1, 'df']
    
    ois1.at[i, 'sum_df'] = sum2 + ois1.at[i-1, 'sum_df']
ois1

,Tenor,Product,Rate,days,years,sum_df,df
0,6m,OIS,0.00250,180,0.5,NaN,NaN
1,1y,OIS,0.00300,360,1.0,0.997009,0.997009
2,2y,OIS,0.00325,720,2.0,1.990540,0.993531
3,3y,OIS,0.00335,1080,3.0,2.980555,0.990015
4,4y,OIS,0.00350,1440,4.0,3.966672,0.986117
5,5y,OIS,0.00360,1800,5.0,4.948856,0.982184
6,7y,OIS,0.00400,2520,7.0,6.898556,0.972406
7,10y,OIS,0.00450,3600,10.0,9.782916,0.955977
8,15y,OIS,0.00500,5400,15.0,14.477704,0.927611
9,20y,OIS,0.00525,7200,20.0,19.033155,0.900076


In [8]:
def equation4(df, rate, years):
    return years*rate*df - (1 - df)
guess = 0.99

ois1.at[0, 'df'] = fsolve(equation4, guess, args=(ois1.at[0, 'Rate'], ois1.at[0, 'years']))[0]
ois1

,Tenor,Product,Rate,days,years,sum_df,df
0,6m,OIS,0.00250,180,0.5,NaN,0.998752
1,1y,OIS,0.00300,360,1.0,0.997009,0.997009
2,2y,OIS,0.00325,720,2.0,1.990540,0.993531
3,3y,OIS,0.00335,1080,3.0,2.980555,0.990015
4,4y,OIS,0.00350,1440,4.0,3.966672,0.986117
5,5y,OIS,0.00360,1800,5.0,4.948856,0.982184
6,7y,OIS,0.00400,2520,7.0,6.898556,0.972406
7,10y,OIS,0.00450,3600,10.0,9.782916,0.955977
8,15y,OIS,0.00500,5400,15.0,14.477704,0.927611
9,20y,OIS,0.00525,7200,20.0,19.033155,0.900076


In [9]:
ois2 = ois1[["years", 'Rate', 'df']].copy()

# Generate the full sequence
full_sequence = np.arange(0.5, 30.5, 0.5)
# Convert 'years' column of your DataFrame to a list
existing_years = ois2['years'].tolist()
# Remove the existing years from the full sequence
new_years = [year for year in full_sequence if year not in existing_years]

additional_years = pd.DataFrame({
    'years': new_years,
    'Rate': [np.nan]*len(new_years),
    'df': [np.nan]*len(new_years)
})
ois2 = pd.concat([additional_years, ois2])
ois2 = ois2.sort_values(by='years')
ois2 = ois2.interpolate(method='linear')

In [10]:
ois2

,years,Rate,df
0,0.5,0.002500,0.998752
1,1.0,0.003000,0.997009
0,1.5,0.003125,0.995270
2,2.0,0.003250,0.993531
1,2.5,0.003300,0.991773
3,3.0,0.003350,0.990015
2,3.5,0.003425,0.988066
4,4.0,0.003500,0.986117
3,4.5,0.003550,0.984150
5,5.0,0.003600,0.982184


# part i q2

In [11]:
irs = pd.read_excel("IR Data.xlsx", sheet_name = 'IRS', usecols = ["Tenor", "Product", "Rate"])
irs["tenor"] = [tenor_to_years(tenor) for tenor in irs["Tenor"]]

irs1 = irs[['tenor', 'Rate']].copy()
# Generate the full sequence
full_sequence = np.arange(0.5, 30.5, 0.5)
# Convert 'years' column of your DataFrame to a list
existing_tenor = irs1['tenor'].tolist()
# Remove the existing years from the full sequence
new_tenor = [tenor for tenor in full_sequence if tenor not in existing_tenor]

additional_tenor = pd.DataFrame({
    'tenor': new_tenor,
    'Rate': [np.nan]*len(new_tenor),
})
irs1 = pd.concat([additional_tenor, irs1])
irs1 = irs1.sort_values(by='tenor')
irs1['ois_df'] = ois2['df']
irs1['libor_df'] = np.nan
irs1 = irs1.reset_index(drop=True)

## method1

In [12]:
def equation1(df, irs):
    irs1 = irs.copy()
    irs1['libor_df'].iloc[-1] = df
    irs1[['tenor', 'libor_df']] = irs1[['tenor', 'libor_df']].interpolate(method='linear')
    sum = 0
    for i in range(len(irs1)):
        if i == 0:
            sum += irs1["ois_df"].iloc[i]/irs1['libor_df'].iloc[i]*2
        else:
            sum += irs1["ois_df"].iloc[i]*irs1['libor_df'].iloc[i-1]/irs1['libor_df'].iloc[i]*2

    return sum - (irs1['Rate'].iloc[-1] + 2)*np.sum(irs1['ois_df'])
guess = 0.2

index = irs1[pd.notna(irs1['Rate'])].index

for i in index:
    irs1.at[i, 'libor_df'] = fsolve(equation1, guess, args=(irs1.iloc[0:i+1]))[0]

## method2

In [13]:
index = irs1[pd.notna(irs1['Rate'])].index

for i in index:
    def equation1(df, i = i):
        irs2 = irs1.iloc[0:i+1].copy()
        irs2['libor_df'].iloc[-1] = df
        irs2[['tenor', 'libor_df']] = irs2[['tenor', 'libor_df']].interpolate(method='linear')
        sum = 0
        for i in range(len(irs2)):
            if i == 0:
                sum += irs2["ois_df"].iloc[i]/irs2['libor_df'].iloc[i]*2
            else:
                sum += irs2["ois_df"].iloc[i]*irs2['libor_df'].iloc[i-1]/irs2['libor_df'].iloc[i]*2
    
        return sum - (irs2['Rate'].iloc[-1] + 2)*np.sum(irs2['ois_df'])

    irs1.at[i, 'libor_df'] = brentq(equation1, 0.001, 1)

irs1 = irs1.interpolate(method='linear')

In [14]:
irs1

,tenor,Rate,ois_df,libor_df
0,0.5,0.025000,0.998752,0.987654
1,1.0,0.028000,0.997009,0.972577
2,1.5,0.029000,0.995270,0.957378
3,2.0,0.030000,0.993531,0.942179
4,2.5,0.030750,0.991773,0.926330
5,3.0,0.031500,0.990015,0.910482
6,3.5,0.032000,0.988066,0.894731
7,4.0,0.032500,0.986117,0.878981
8,4.5,0.032750,0.984150,0.863985
9,5.0,0.033000,0.982184,0.848989


# part i q3

In [15]:
fsr = pd.DataFrame({'start' : [1]*5 + [5]*5 + [10]*5,'tenor' : [1,2,3,5,10]*3, 'fsr': [np.nan]*15})

for i in fsr.index:
    def equation(S, start = fsr.at[i, 'start'], tenor = fsr.at[i, 'tenor'], irs = irs1):
        irs1 = irs[(irs['tenor'] > start)&(irs['tenor'] <= start+tenor)].copy()
    
        sum1 = 0
        for i in irs1.index:
                sum1 += irs.at[i, 'ois_df']*irs.at[i-1, 'libor_df']/irs.at[i, 'libor_df']*2
        
        sum2 = (S+2)*np.sum(irs1['ois_df'])
        
        return sum1-sum2

    fsr.at[i, 'fsr'] = brentq(equation, 0.0001, 0.5)

In [20]:
fsr

,start,tenor,fsr
0,1,1,0.032007
1,1,2,0.033259
2,1,3,0.034011
3,1,5,0.035255
4,1,10,0.038428
5,5,1,0.039274
6,5,2,0.040075
7,5,3,0.040072
8,5,5,0.041093
9,5,10,0.043634


In [23]:
ois2

,years,Rate,df
0,0.5,0.002500,0.998752
1,1.0,0.003000,0.997009
0,1.5,0.003125,0.995270
2,2.0,0.003250,0.993531
1,2.5,0.003300,0.991773
3,3.0,0.003350,0.990015
2,3.5,0.003425,0.988066
4,4.0,0.003500,0.986117
3,4.5,0.003550,0.984150
5,5.0,0.003600,0.982184


In [29]:
### cal PVBP for part2
for i in fsr.index:
    expiry = fsr.at[i, 'start']
    tenor = fsr.at[i, 'tenor']
    ois3 = ois2[(ois2['years'] > expiry) & (ois2['years'] <= (expiry + tenor))].copy()

    fsr.at[i, 'PVBP'] = np.sum(ois3['df'])*0.5

In [30]:
fsr

,start,tenor,fsr,PVBP
0,1,1,0.032007,0.994400
1,1,2,0.033259,1.985294
2,1,3,0.034011,2.972386
3,1,5,0.035255,4.934070
4,1,10,0.038428,9.747887
5,5,1,0.039274,0.978517
6,5,2,0.040075,1.952145
7,5,3,0.040072,2.920444
8,5,5,0.041093,4.840612
9,5,10,0.043634,9.542492


In [31]:
fsr.to_csv('fsr.csv', index = False)